# 1 : Import required dependencies

In [1]:
import pandas as pd
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

# 2: Discover your data

In [2]:
from google.colab  import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
data = pd.read_csv("/content/drive/MyDrive/ITI/preprocessed_data.csv")
data = data.dropna()

take samples to train ur model on it at the first time

In [5]:
sampled_data = data.sample(n=5000, random_state=42)

In [6]:
sampled_data.head()

,Unnamed: 0,text,dialect
25652,25652,يمكن تستغرب شء,2
52025,52025,خلصتي ولسه بتسقطي دي هوايه عندك بقي,0
5368,5368,صارت معايا مره,2
37970,37970,ومال تماره راه اغلب كانو الرباط شرو تماره بحكم...,3
117693,117693,الله يلعن السياسه يللي بتبعد الشرفاء بالبلد هي...,1


In [7]:
data["dialect"].shape

(147642,)

#2: Text representation

### 1: TF-IDF
tf-idf is the preferred method in this situation since it evaluates the sentence as a complete entity rather than focusing on individual words, and takes into account the significance of each word based on its frequency of occurrence rather than its contextual relevance.

In [8]:
vectorizer = TfidfVectorizer(lowercase=True)
X = vectorizer.fit_transform(data["text"])

### 2:word2vec

In [9]:
## tokenization
tokenized_sentences = []
for sentence in sampled_data['text']:
    tokens = nltk.word_tokenize(sentence)
    tokenized_sentences.append(tokens)

In [10]:
## word2vec
from gensim.models import Word2Vec
model = Word2Vec(tokenized_sentences, vector_size=100, window=5, min_count=1, workers=4)
sentence_vectors = []
for tokens in tokenized_sentences:
    vector = sum([model.wv[word] for word in tokens]) / len(tokens)
    sentence_vectors.append(vector)

#3: MODEL





In [11]:
# Split Data
from sklearn.model_selection import train_test_split
train_X, test_X, train_y, test_y = train_test_split(data["text"], data['dialect'], test_size=0.2, random_state=42)

### SVC
It took 67 minutes to train the entire dataset.


In [12]:
# make the pipline vectorization>>>model
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC

model = SVC(kernel='linear', C=1, gamma='auto')

pipeline = Pipeline([
    ('tfidf',  TfidfVectorizer()),
    ('svc', model)
])
pipeline.fit(train_X, train_y)

Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('svc', SVC(C=1, gamma='auto', kernel='linear'))])

## Lightgbm

In [13]:
from sklearn.pipeline import Pipeline
import lightgbm as lgb
model = lgb.LGBMClassifier()

lightgbm_pipeline = Pipeline([
    ('tfidf',  TfidfVectorizer()),

    ('lightgbm', model)
])
lightgbm_pipeline.fit(train_X, train_y)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 13.554514 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 244294
[LightGBM] [Info] Number of data points in the train set: 118113, number of used features: 8331
[LightGBM] [Info] Start training from score -0.940981
[LightGBM] [Info] Start training from score -1.680236
[LightGBM] [Info] Start training from score -1.396549
[LightGBM] [Info] Start training from score -2.548208
[LightGBM] [Info] Start training from score -2.325303


Pipeline(steps=[('tfidf', TfidfVectorizer()), ('lightgbm', LGBMClassifier())])

#4: Evaluation

## 1: evaluate SVC

In [14]:
# evaluation
from sklearn.metrics import classification_report
predictions = pipeline.predict(test_X)

In [15]:
print(classification_report(test_y, predictions))

              precision    recall  f1-score   support

           0       0.82      0.92      0.86     11525
           1       0.86      0.82      0.84      5607
           2       0.78      0.82      0.80      7251
           3       0.90      0.64      0.75      2298
           4       0.85      0.57      0.68      2848

    accuracy                           0.82     29529
   macro avg       0.84      0.75      0.79     29529
weighted avg       0.82      0.82      0.82     29529



## 2: Evaluate LIGHTGBM
it took about 2 minutes in whole dataset but the result in production less accurate than svc

In [16]:
# evaluation
lightgbm_predictions = lightgbm_pipeline.predict(test_X)

In [17]:
print(classification_report(test_y, lightgbm_predictions))

              precision    recall  f1-score   support

           0       0.77      0.87      0.82     11525
           1       0.86      0.69      0.77      5607
           2       0.62      0.77      0.69      7251
           3       0.88      0.56      0.68      2298
           4       0.82      0.48      0.60      2848

    accuracy                           0.75     29529
   macro avg       0.79      0.67      0.71     29529
weighted avg       0.77      0.75      0.75     29529



#5: Save the model

In [18]:
import pickle
pickle.dump(pipeline ,open("/content/drive/MyDrive/ITI/SVC_model.pkl","wb"))

test saved model

In [19]:
def dialect_predict(model_path,sentence):
  model = pickle.load(open(model_path,"rb"))
  sentence =[sentence]
  dialect = model.predict(sentence)

  if dialect == 0:
    country = "Egypt"

  elif  dialect == 1 :
    country = "Lebnanon"

  elif dialect ==   2 :
    country = "Libya"

  elif dialect == 3:
    country = "Morocco"

  else:
    country = "Sudan"

  return country

In [20]:
model_path = "/content/drive/MyDrive/ITI/SVC_model.pkl"

In [45]:
dialect_predict(model_path,"عامل ايه دلوقت")

'Egypt'

In [36]:
dialect_predict(model_path,"هنط جمب بالاوكابى")

'Egypt'

In [38]:
dialect_predict(model_path,"شو بتعمل")

'Lebnanon'

In [44]:
dialect_predict(model_path," بالزاف ")

'Libya'

In [41]:
dialect_predict(model_path,"باهى باهى")

'Libya'

In [42]:
dialect_predict(model_path,"يا الزول")

'Sudan'